In [74]:
!pip install xmltodict


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [75]:
import platform as python_platform

class Platform:
    WINDOWS = "Windows"
    MACOS = "Darwin"

platform = python_platform.system()

windows_root_directory = "D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling"
mac_root_directory = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling"

root_directory = windows_root_directory if platform == Platform.WINDOWS else mac_root_directory

In [76]:
import os
from os.path import join, getsize

import sys
sys.path.append(root_directory)

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
# from requests_html import HTMLSession

import random
import time
import re

import xmltodict

import os
import json
import threading

import file_utils as fu

In [77]:
class ResponseType:
    JSON = "json"
    XML = "xml"
    UNKNOWN = "unknown"

In [78]:
def copy_folder_structure(source, destination):
    '''Copy the structure of the source directory to the destination directory'''
    for root, dirs, files in os.walk(source):
        for dir in dirs:
            new_dir = f"{destination}/{root.replace(source, '')}/{dir}"
            
            regex_pattern = r"\/{2,}"            
            new_dir = re.sub(regex_pattern, "/", new_dir)

            os.makedirs(new_dir, exist_ok=True)

In [79]:
song_ids_of_song_parts_dir = fu.relative_to_absolute_path(
    "DataPreprocessing/song_id_of_song_parts",
    root_path=root_directory
)

song_parts_from_ids_dir = fu.relative_to_absolute_path(
    "DataPreprocessing/song_parts_from_ids",
    root_path=root_directory
)

copy_folder_structure(
    source=song_ids_of_song_parts_dir,
    destination=song_parts_from_ids_dir
)

In [80]:
def response_data_parser(response_json):
    metadata = None
    main_data_type = None
    main_data = None

    if "jsonData" in response_json and response_json["jsonData"] is not None:
        main_data_type = ResponseType.JSON
        main_data = response_json["jsonData"]
    elif "xmlData" in response_json and response_json["xmlData"] is not None:
        main_data_type = ResponseType.XML
        main_data = xmltodict.parse(response_json["xmlData"])
    else:
        main_data_type = ResponseType.UNKNOWN
        main_data = response_json

        print(f"WARNING: response_data_parser: Unknown response type: {response_json}")

    try:
        del response_json["jsonData"]
        del response_json["xmlData"]
    except:
        print(f"WARNING: response_data_parser: Unable to delete jsonData or xmlData from response_json: {response_json}")

    metadata = response_json

    return metadata, main_data_type, main_data

In [81]:
def make_request(url, parameters: dict = None, headers: dict = None):
    session = Session()
    session.headers.update(headers)
    
    status_code = None
    metadata = None
    main_data_type = None
    main_data = None
    
    try:
        response = session.get(url, params=parameters)
        status_code = response.status_code

        if status_code != 200:
            print(f"-----------> FAIL: make_request - status_code: {status_code}")
        else:        
            if response.headers['Content-Type'] == 'application/json; charset=UTF-8':
                metadata, main_data_type, main_data = response_data_parser(response.json())
            else:
                print(f"-----------> FAIL: make_request - unknown format: {url}")
        
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

    return {
        'status_code' : status_code,
        'metadata' : metadata,
        'main_data_type' : main_data_type,
        'main_data' : main_data # Tương ứng với phần jsonData hoặc xmlData
    }

In [ ]:
lock = threading.Lock()

thread_per_batch = 10

thread_list = []

In [ ]:
def make_request_in_thread_mode(song_part_url, song_url):
    # Add join to all the threads that are still running
    for thread in thread_list:
        thread.join()

    while threading.active_count() > thread_per_batch:
        # Random sleep time from 1000 to 2000 milliseconds
        time_to_sleep = random.randint(1000, 2000) / 1000
        time.sleep(time_to_sleep)
    
    current_number_of_threads = threading.active_count()
    if (current_number_of_threads > thread_per_batch):
        print(f"WARNING: current_number_of_threads: {current_number_of_threads} - {song_part_url}")
    
    thread = threading.Thread(target=make_request, args=song_part_url)
    thread.start()

    result = thread.result

    

    return request_result

In [86]:
json_url = "https://api.hooktheory.com/v1/songs/public/jDgXpWlLgKl"
xml_url = "https://api.hooktheory.com/v1/songs/public/lamknK_AoDM"
xml_url_2 = "https://api.hooktheory.com/v1/songs/public/ZwxKJrPGged"

test_url = xml_url
test_url = json_url
test_url = xml_url_2

result = make_request(test_url)
print(f"status_code: {result['status_code']}")
print(f"metadata: {result['metadata']}")
print(f"main_data_type: {result['main_data_type']}")
print(f"main_data: {result['data']}")

status_code: 200
metadata: {'ID': 133315, 'user_id': 103696, 'dateModified': '2018-07-31 12:44:48+00:00', 'HTID': 'ogmC3LVRvUiD', 'artist': 'Armin van Buuren', 'song': 'This Is What It Feels Like', 'section': 'Chorus', 'revision': 1, 'chords': None, 'youTubeID': 'MS1a-SQd79w', 'dotDNAbin': None, 'xmlType': 'file', 'SIF': ',6-4,4-2,1-6,56-2,6-4,4-2,1-6,36-2,6-4,4-2,1-8,5-2,6-2,4-2,1-8,', 'SInE': ',6-4,4-2,1-6,56-2,6-4,4-2,1-6,36-2,6-4,4-2,1-8,5-2,6-2,4-2,1-8,', 'SInD': ',6,4,1,56,6,4,1,36,6,4,1,5,6,4,1,', 'SInED': ',6,4,1,56,6,4,1,36,6,4,1,5,6,4,1,', 'SInA': ',6,4,1,5,6,4,1,3,6,4,1,5,6,4,1,', 'beatsInMeasure': '4', 'songKey': 'F', 'isPrivate': '1', 'views': 0, 'hasMelody': '1', 'hasChords': '1', 'isAnalysis': 1, 'voteup': 0, 'votedown': 0, 'score': 0, 'songURL': 'this-is-what-it-feels-like', 'artistURL': 'armin-van-buuren', 'sectionURL': 'chorus', 'editComments': 'Fixed the synchronization.', 'visible': 1, 'wikiorder': 1, 'bIsPaired': 0, 'YouTubeVideoID': None, 'bpm': 128, 'mode': 6, 'b

In [83]:
# {
#     "artist_name": "ZZ Top",
#     "artist_link": "https://www.hooktheory.com/theorytab/artists/z/zz-top",
#     "song_name": "Sharp Dressed Man",
#     "link": "https://www.hooktheory.com/theorytab/view/zz-top/sharp-dressed-man",
#     "metrics": {
#         "Chord Complexity": "26",
#         "Melodic Complexity": "30",
#         "Chord-Melody Tension": "95",
#         "Chord Progression Novelty": "27",
#         "Chord Bass Melody": "38"
#     },
#     "metrics_conclusion": " Sharp Dressed Man has higher complexity than the average song  in terms Chord-Melody Tension. ",
#     "key_conclusion": " Sharp Dressed Man is written in the key of C Dorian.  \n\nAccording to the Theorytab database, it is the 4th most popular key among Dorian keys and     the 33rd most popular  among all keys. The C Dorian scale is similar to the C Minor scale except that its 6th note is a half step higher (A).  Chord progressions in Dorian have a characteristic sound due to the major quality of the chord built on the 4th scale degree. \n\nSee the C Dorian Cheat Sheet for popular chords, chord progressions, downloadable midi files and more!",
#     "song_parts": [
#         {
#             "id" : "nJmB_pZdgAV",
#             "metadata": {...<json-data>...},
#             "main_data_type": "xml",
#             "json_path": "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids/ZZ Top/<song-name>/<id>.json"
#         },
#         {
#             ...
#         },
#         ...
#     ]
# }


def aggregate_song_parts_data(song_parts_json_response_list, song_id_of_song_parts_dict: dict):
    '''Gom nhóm dữ liệu của các phần nhạc của một bài hát'''
    
    song_parts_data = []
    threads = []
    
    for song_part_json_response in song_parts_json_response_list:
        song_part_id = song_part_json_response['id']
        song_part_metadata = song_part_json_response['metadata']
        
        song_part_data = {
            'id' : song_part_id,
            'metadata' : song_part_metadata,
            'json_path' : song_id_of_song_parts_dict[song_part_id]
        }
        
        song_parts_data.append(song_part_data)
    
    # Thay key "song_parts_ids" trong song_parts_json_response_list thành "song_parts"
    song_id_of_song_parts_dict["song_parts"] = song_parts_data
    del song_id_of_song_parts_dict["song_parts_ids"]

    return song_id_of_song_parts_dict
    

In [92]:
# Ví dụ về threading và callback 

def callback_function(result):
    print(f"callback_function: {result}")

def calculate_square(number, callback):
    time.sleep(number)
    result = number * number
    callback(result)

def calculate_cube(number, callback):
    time.sleep(1)
    result = number * number * number
    callback(result)

thread1 = threading.Thread(target=calculate_square, args=(5, callback_function))
thread1.start()

thread2 = threading.Thread(target=calculate_cube, args=(10, callback_function))
thread2.start()

thread1.join()
thread2.join()  

callback_function: 1000
callback_function: 25


In [84]:
test_file_path = '/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids/test.json'
# Lưu data vào file json
with open(test_file_path, 'w') as file:
    file.write(result['data'])

print(test_file_path)


/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids/test.json


In [85]:
print(type(result['data']))

x = json.loads(result['data'])
print(type(x))

print(f"jsonData: {x['jsonData']}")

z = xmltodict.parse(x['xmlData'])
print(type(z))

y = json.dumps(xmltodict.parse(x['xmlData']))
print(type(y))

print(y)



<class 'str'>
<class 'dict'>


KeyError: 'jsonData'

In [ ]:
t = {"a" : 1, "b" : 2}
print(t)

k = t.pop("g")
print(k)
print(t)